In [1]:
setwd('/Users/alexis/IEHS Dropbox/Rager Lab/Alexis_Payton/2_Wildfire_Analysis/3_Statistics/Input')
Output = ('/Users/alexis/IEHS Dropbox/Rager Lab/Alexis_Payton/2_Wildfire_Analysis/3_Statistics/Output')
cur_date = '020421'

library(readxl)
library(tidyverse)
library(reshape2)
library(GGally)
library(ggplot2)
library(itertools)

#reading in file
counts_df = read_csv("010521_NormCounts_pslog2_.csv")
sampleinfo = read_csv("Sample_Info_112520.csv")

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.6     ✔ dplyr   1.0.4
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2

Loading required package: iterators

Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_double(),
  X1 = col_character()
)
ℹ Use `spec()` for the full column specifications.



── Column specification ────────────────────────────────────────────────────────
cols(
  SampleID_BioSpyderCountFile = col_character(),
  Plat

In [2]:
head(counts_df)

X1,M1_RedOakSmolder,M2_RedOakSmolder,M3_RedOakSmolder,M4_RedOakSmolder,M5_RedOakSmolder,M6_RedOakSmolder,M7_PeatSmolder,M8_PeatSmolder,M9_PeatSmolder,⋯,MH73_RedOakSmolder,MH74_RedOakSmolder,MH75_RedOakSmolder,MH76_RedOakSmolder,MH77_PeatSmolder,MH78_PeatSmolder,MH79_PeatSmolder,MH80_PeatSmolder,MH81_PeatSmolder,MH82_PeatSmolder
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Bcat1_29039,0.000000,0.0000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000000,⋯,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Laptm5_29040,0.000000,0.8162391,2.151526,0.000000,0.000000,0.000000,1.310669,0.000000,0.8440234,⋯,0.000000,0.000000,0.000000,2.044687,0.000000,0.000000,0.000000,0.000000,0.000000,1.460427
Skp2_29041,0.000000,0.0000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000000,⋯,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Nme1_29042,9.496876,9.4182895,9.482641,9.411011,9.680811,9.421780,9.155520,9.517998,9.2165078,⋯,10.390459,9.717889,10.197414,9.690385,9.958532,9.541203,9.594351,8.854131,9.858309,9.476282
Ctsd_29043,5.477012,3.8005134,4.043816,4.175141,4.833601,4.048264,3.913164,4.974692,4.9640230,⋯,5.171545,4.766944,5.079045,4.516070,5.459466,5.242892,5.720632,5.732964,5.081831,5.485345
Camk1_29044,7.749153,9.1892817,9.053160,8.127899,8.723191,9.020838,8.318866,8.358731,8.7913172,⋯,11.195259,10.882216,11.223821,10.357870,10.624620,10.763880,10.477399,10.427105,10.528044,10.422839


In [3]:
head(sampleinfo)

SampleID_BioSpyderCountFile,PlateBatch,MouseID,NumericID,Treatment,ID,Timepoint,Tissue,Group
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Plate1-m1-RNA,Plate1,M1,1,RedOakSmolder,M1_RedOakSmolder,4h,Lung,RedOakSmolder_4h_Lung
Plate1-m2-RNA,Plate1,M2,2,RedOakSmolder,M2_RedOakSmolder,4h,Lung,RedOakSmolder_4h_Lung
Plate1-m3-RNA,Plate1,M3,3,RedOakSmolder,M3_RedOakSmolder,4h,Lung,RedOakSmolder_4h_Lung
Plate1-m4-RNA,Plate1,M4,4,RedOakSmolder,M4_RedOakSmolder,4h,Lung,RedOakSmolder_4h_Lung
Plate1-m5-RNA,Plate1,M5,5,RedOakSmolder,M5_RedOakSmolder,4h,Lung,RedOakSmolder_4h_Lung
Plate1-m6-RNA,Plate1,M6,6,RedOakSmolder,M6_RedOakSmolder,4h,Lung,RedOakSmolder_4h_Lung


In [4]:
#renaming first col
names(counts_df)[1] = "Gene"

#removing heart tissue samples
counts_df = counts_df[,colnames(counts_df)[1:140]]

#need to melt df so I can separate the sample ids and get timepoints from other file
melted_counts_df = melt(counts_df, variable = 'SampleID', value.name = 'NormCount') 
head(melted_counts_df)

Using Gene as id variables



,Gene,SampleID,NormCount
,<chr>,<fct>,<dbl>
1,Bcat1_29039,M1_RedOakSmolder,0.000000
2,Laptm5_29040,M1_RedOakSmolder,0.000000
3,Skp2_29041,M1_RedOakSmolder,0.000000
4,Nme1_29042,M1_RedOakSmolder,9.496876
5,Ctsd_29043,M1_RedOakSmolder,5.477012
6,Camk1_29044,M1_RedOakSmolder,7.749153


In [5]:
#inner joining with subject info to get the time points/tissue of the samples
melted_counts_df = melted_counts_df %>%
    inner_join(sampleinfo, by = c("SampleID" = "ID")) %>%
    select(Gene, SampleID, NormCount, Timepoint) %>%
    separate(SampleID, c(NA, "Exposure")) #don't the first part of the id
head(melted_counts_df)

,Gene,Exposure,NormCount,Timepoint
,<chr>,<chr>,<dbl>,<chr>
1,Bcat1_29039,RedOakSmolder,0.000000,4h
2,Laptm5_29040,RedOakSmolder,0.000000,4h
3,Skp2_29041,RedOakSmolder,0.000000,4h
4,Nme1_29042,RedOakSmolder,9.496876,4h
5,Ctsd_29043,RedOakSmolder,5.477012,4h
6,Camk1_29044,RedOakSmolder,7.749153,4h


In [6]:
#now taking an average of normalized counts for each gene based on its exposure and timepoint
avg_count_df = melted_counts_df %>%
    group_by(Timepoint, Exposure, Gene) %>%
    summarize(Avg_NormCount = mean(NormCount))
head(avg_count_df)

`summarise()` has grouped output by 'Timepoint', 'Exposure'. You can override using the `.groups` argument.



Timepoint,Exposure,Gene,Avg_NormCount
<chr>,<chr>,<chr>,<dbl>
24h,EucalyptusFlame,0610009B22Rik_56046,5.2164306
24h,EucalyptusFlame,0610010F05Rik_69119,10.0883485
24h,EucalyptusFlame,0610010F05Rik_74637,3.8062197
24h,EucalyptusFlame,0610010F05Rik_77250,0.9262457
24h,EucalyptusFlame,0610010K14Rik_31619,9.4314208
24h,EucalyptusFlame,0610010K14Rik_31873,9.3811087


In [7]:
treatment = c('EucalyptusFlame', 'EucalyptusSmolder','LPS', 'PeatFlame','PeatSmolder','PineFlame','PineNeedlesFlame', 
             'PineNeedlesSmolder', 'PineSmolder', 'RedOakFlame', 'RedOakSmolder')
combinations = data.frame(t(combn(treatment, 2)))
colnames(combinations) = c("Treatment1", "Treatment2")
head(combinations)

,Treatment1,Treatment2
,<fct>,<fct>
1,EucalyptusFlame,EucalyptusSmolder
2,EucalyptusFlame,LPS
3,EucalyptusFlame,PeatFlame
4,EucalyptusFlame,PeatSmolder
5,EucalyptusFlame,PineFlame
6,EucalyptusFlame,PineNeedlesFlame


In [8]:
#now running pearson's correlation coefficients
time = c("4h", "24h")
empty_vector = c()

for (i in 1:length(time)){
    for (j in 1:length(combinations$Treatment1)){
        #creating two df to compare col of averages 
        treatment_1_df = avg_count_df %>%
            filter(Timepoint == time[i], Exposure == combinations$Treatment1[j])
        treatment_2_df = avg_count_df %>%
            filter(Timepoint == time[i], Exposure == combinations$Treatment2[j])
        
      
        #calculating pearson's coefficient
        pearson_corr = cor.test(treatment_1_df$Avg_NormCount, treatment_2_df$Avg_NormCount, method = c("pearson"))

        #vector of values containing, the timepoint, treatment 1, treatment 2, coeff, and p value
        values_vector = c(time[i], as.character(combinations$Treatment1[j]), as.character(combinations$Treatment2[j]),
                          pearson_corr$estimate, pearson_corr$p.value)
        empty_vector = c(empty_vector, values_vector)

    }
}

In [9]:
final_table = function(vector){
    #"""
    #Reformatting each vector into matrices to export. 
    
    #:param: vector, demographic variable
    #:output: a 5x110 matrix containing time, tx 1, tx 2, estimate, p value, p adj

    #"""
    dim(vector) = c(5, 110)
    table = data.frame(t(vector))    
    colnames(table) = c('Time', 'Treatment1', 'Treatment2', 'Coefficient', 'P Value')
    return(table)
}

#calling fn
pearson_table = final_table(empty_vector)
head(pearson_table)

,Time,Treatment1,Treatment2,Coefficient,P Value
,<fct>,<fct>,<fct>,<fct>,<fct>
1,4h,EucalyptusFlame,EucalyptusSmolder,0.992523696845208,0
2,4h,EucalyptusFlame,LPS,0.971895587418455,0
3,4h,EucalyptusFlame,PeatFlame,0.99124601594519,0
4,4h,EucalyptusFlame,PeatSmolder,0.988060448645687,0
5,4h,EucalyptusFlame,PineFlame,0.99174379570422,0
6,4h,EucalyptusFlame,PineNeedlesFlame,0.992857014822994,0


In [10]:
#exporting
write.csv(pearson_table, paste0(Output,"/", cur_date, "_mRNA_Treatment_Correlations.csv"), row.names = FALSE)